Домашняя работа 1  
Зенина Алёна БКЛ-201
# 1) Сбор данных
В качестве источника данных использован сайт [otziv-otziv.ru](otziv-otziv.ru), а точнее его раздел с отзывами на различные вина

In [1]:
import requests
from bs4 import BeautifulSoup

Собираю все ссылки на страницу каждого вина с общей страницы каталога отзывов

In [2]:
vine_page = [] 

for i in range(1, 107):
    req = 'https://otziv-otziv.ru/katalog/vino/?page=' + str(i)
    response = requests.get(req)
    
    if (response.status_code == 200):
        soup = BeautifulSoup(response.text, 'html.parser')
        page = soup.findAll('div', {'class': 'content'})
        
        for block in page:
            vine_page.append(block.find('a')['href'])
            
    else:
        print(i, ' error')

In [3]:
len(vine_page)

2332

Собираю положительные (оценка 4, 5) и отрицательные (оценка 1, 2) отзывы на вина  
Не беру отзывы, где стоит оценка 3, потому что она по идее тонально не окрашена  
*Warning! следующий кусок работает ооочень долго (около получаса)*

In [6]:
pos_global = []
neg_global= []

for i in vine_page:
    req = 'https://otziv-otziv.ru' + i
    response = requests.get(req)
    
    if (response.status_code == 200):
        soup = BeautifulSoup(response.text, 'html.parser')
        review = soup.findAll('div', {'class': 'container-reviews'})

        for block in review:
            rating = int(block.find('div', {'class': 'stars-container'}).get('title')[-1])    
            text = block.get_text()[24:]
    
            if rating > 3:
                pos_global.append(text)
                
            elif rating < 3:
                neg_global.append(text)
     
    else:
        print('error')

In [7]:
print('Положительных отзывов: ', len(pos_global), '\nОтрицательных отзывов:', len(neg_global))

Положительных отзывов:  4003 
Отрицательных отзывов: 746


# 2) Предпроцессинг
Выборка по винам получилась несбалансированной, поэтому, пытаюсь ее уравновесить, выбирая из большего массива количество отзывов равное величине меньшего (в нашем случае положительных больше)

In [8]:
import numpy as np
import random

In [9]:
def here(posx, negx):

    if len(posx) > len(negx):
        a = random.choices(np.array(posx), k=min(len(posx), len(negx)))
        b = negx
    
    elif len(posx) < len(negx):
        a = posx
        b = random.choices(np.array(negx), k=min(len(posx), len(negx)))
    
    else:
        a = posx
        b = negx
        
    return a, b

In [10]:
pos, neg = here(pos_global, neg_global)
print('Положительных отзывов: ', len(pos), '\nОтрицательных отзывов:', len(neg))

Положительных отзывов:  746 
Отрицательных отзывов: 746


Итого общая выборка получается объемом 1492 отзыва, примерно одну десятую оставляю на проверку качества

In [11]:
sample = []
r = round(min(len(pos), len(neg))/10)

if r%2 == 1:
    r -= 1
    
for i in pos[0: r]:
    sample.append(i)

for i in neg[0: r]:
    sample.append(i)
    
del neg[0: r]
del pos[0: r] 

Делаю массив из 1 и -1, характеризующих тональность отзыва (1 = положительный, -1 = отрицательный), для дальнейшей проверки точности

In [12]:
y = []

for i in range(0, r):
    y.append(1)

for i in range(0, r):
    y.append(-1)

Токенизирую слова, привожу к нижнему регистру и к начальной форме

In [13]:
import nltk
import pymorphy2
morph = pymorphy2.MorphAnalyzer()

In [14]:
def tokenize_list(lst):
    all_words = []
    
    for comment in lst:
        tokens = nltk.word_tokenize(comment)
        words = [word.lower() for word in tokens if word.isalpha()]
        all_words.extend(lemmatize(words))
    
    return all_words

def tokenize_string(strg):
    tokens = nltk.word_tokenize(strg)
    words = [word.lower() for word in tokens if word.isalpha()]
    lemmas = lemmatize(words)
    
    return lemmas

def lemmatize(lst):
    res = []
    
    for word in lst:
        p = morph.parse(word)[0]
        res.append(p.normal_form)
        
    return res

In [15]:
tokenized_neg = tokenize_list(neg)
tokenized_pos = tokenize_list(pos)

# 3) Составление словарей
Создаю частотный словарь


In [16]:
from collections import Counter

In [17]:
counted_neg = Counter(tokenized_neg)
counted_pos = Counter(tokenized_pos)

Ввожу функцию, которая убирает из частотного словаря слова, которые встретились меньше определенного количества раз и убирает значения частотности

In [18]:
def frequently(dct, n):
    words = np.array([i for i in dct.keys()])
    frequency = np.array([i for i in dct.values()])
    res = set(words[frequency > n])

    return res

In [19]:
for i in range (0,7):
    print('i =', i, ' Количество слов в +: ', len(frequently(counted_pos, i)), ' Количество слов в -: ', len(frequently(counted_neg, i)))

i = 0  Количество слов в +:  1614  Количество слов в -:  1904
i = 1  Количество слов в +:  785  Количество слов в -:  812
i = 2  Количество слов в +:  508  Количество слов в -:  521
i = 3  Количество слов в +:  395  Количество слов в -:  400
i = 4  Количество слов в +:  327  Количество слов в -:  323
i = 5  Количество слов в +:  269  Количество слов в -:  275
i = 6  Количество слов в +:  230  Количество слов в -:  235


Далее буду рассматривать вариант, что частотность больше 2, но этот параметр можно поменять и возможно, получится другой конечный результат (об этом далее в разделе 5)  
Убираю пересекающиеся в обоих словарях слова

In [20]:
unique_neg = frequently(counted_neg, 2).difference(frequently(counted_pos, 2))
unique_pos = frequently(counted_pos, 2).difference(frequently(counted_neg, 2))

# 4) Положительный или отрицательный?
Ввожу функцию опредления тональности, которая выдает такую тональность, к словарю которой относится больше слов в выбранном отзыве  
Если в функции слов из обоих словарей будет одинаково, то будем считать его положительным, поскольку их больше (опытным путем выяснено, что это улучшает результат на 20+%)

In [21]:
def tone(test, posx, negx):
    y_pred = []
    
    for text in test:
        val = 0
        tokenized_text = tokenize_string(text)
        
        for token in tokenized_text:
            
            if token in posx:
                val += 1
            
            if token in negx: 
                val -= 1
        
        if val >= 0: 
            y_pred.append(1)
        
        if val < 0:
            y_pred.append(-1)
        
            
    return y_pred

In [22]:
from sklearn.metrics import accuracy_score

In [23]:
accuracy_score(y, tone(sample, unique_pos, unique_neg))

0.7837837837837838

Неплохо! Но будем пытаться улучшать!

# 5) Улучшения
### 5.1) Окно
**Первое**, что пришло в голову, это поменять допустимое окно частотности слов

In [24]:
for i in range (0,7):
    print('i =', i, ' Accuracy ', accuracy_score(y, tone(sample, frequently(counted_pos, i).difference(frequently(counted_neg, i)), frequently(counted_neg, i).difference(frequently(counted_pos, i)))))

i = 0  Accuracy  0.7567567567567568
i = 1  Accuracy  0.7702702702702703
i = 2  Accuracy  0.7837837837837838
i = 3  Accuracy  0.7837837837837838
i = 4  Accuracy  0.7635135135135135
i = 5  Accuracy  0.75
i = 6  Accuracy  0.722972972972973


Предположительно лучшее окно - 2 (так как при каждом новом запуске программы, тексты немного меняются, могу сказать, что окно два дает лучшие результаты)
### 5.2) Стоп-слова
**Второе**, убрать стоп-слова

In [25]:
from nltk.corpus import stopwords

In [26]:
stops = stopwords.words('russian')

In [27]:
stops_pos = unique_pos.difference(stops)
stops_neg = unique_neg.difference(stops)
print( 'Положительных со стоп-словами ', len(unique_pos), 'без стоп-слов ', len(stops_pos), '\nОтрицательных со стоп-словами ', len(unique_neg), 'без стоп-слов ', len(stops_neg))

Положительных со стоп-словами  211 без стоп-слов  205 
Отрицательных со стоп-словами  224 без стоп-слов  210


In [28]:
for i in range (0,7):
    print('i =', i, ' Accuracy ', accuracy_score(y, tone(sample, frequently(counted_pos, i).difference(frequently(counted_neg, i)).difference(stops), frequently(counted_neg, i).difference(frequently(counted_pos, i)).difference(stops))))

i = 0  Accuracy  0.7432432432432432
i = 1  Accuracy  0.7702702702702703
i = 2  Accuracy  0.7972972972972973
i = 3  Accuracy  0.7635135135135135
i = 4  Accuracy  0.75
i = 5  Accuracy  0.722972972972973
i = 6  Accuracy  0.722972972972973


Непонятно, лучше или хуже в целом, так как конкретно в это случае со стоп-словами получилось лучше, а в предыдущий раз нет, но по идее в стоп-словах есть слова, обозначающие противопоставление и отрицание, которые важны для отрицательных отзывов 
### 5.3) Учитывать ранг слова (спойлер! не вышло ничего хорошего)
**Третье**, попытаться присвоить значения ранга каждому слову в зависимости от частоты встречаемости: чем чаще, тем больше важности  
Меняю функцию *frequently* на ее аналог, выдающий словарь

In [29]:
def often(dct, n):
    new_dct = {}
    words = np.array([i for i in dct.keys()])
    
    for word in words:
        
        if dct[word] > n:
            new_dct[word] = dct[word]

    return new_dct

Так как теперь работа идет со словарями, очищать от пересекающейся лексики их придется отдельной функцией

In [30]:
def clear(dct_pos, dct_neg):
    new_pos = {}
    new_neg = {}
    words_pos = np.array([i for i in dct_pos.keys()])
    words_neg = np.array([i for i in dct_neg.keys()])
    
    for word1 in words_pos:
        
        if word1 not in words_neg:
            new_pos[word1] = dct_pos[word1]
            
    for word2 in words_neg:
        
        if word2 not in words_neg:
            new_pos[word2] = dct_pos[word2]
            
    return new_pos, new_neg   

In [31]:
some_pos, some_neg = clear(counted_pos, counted_neg)

Я пыталась подобрать функцию, которая бы связывала частотность и вклад в тональность, но что-то в целом тут не так, поэтому что есть

In [32]:
def tone_upgrade(test, posx, negx):
    y_pred = []
    pos_words = np.array([i for i in posx.keys()])
    neg_words = np.array([i for i in negx.keys()])
    
    for text in test:
        val = 0
        tokenized_text = tokenize_string(text)
        
        for token in tokenized_text:
            
            if token in pos_words:
                val += 1.1**int(posx[token])
            
            if token in neg_words: 
                val -= 1.1**int(negx[token])
        
        if val >= 0: 
            y_pred.append(1)
        
        if val < 0:
            y_pred.append(-1)
            
    return y_pred

In [33]:
for i in range (2,7):
    print('i =', i, ' Accuracy ', accuracy_score(y, tone_upgrade(sample, often(some_pos, i), often(some_neg, i))))

<ipython-input-32-3917d4d5ed91>:15: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if token in neg_words:


i = 2  Accuracy  0.5


<ipython-input-32-3917d4d5ed91>:15: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if token in neg_words:


i = 3  Accuracy  0.5


<ipython-input-32-3917d4d5ed91>:15: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if token in neg_words:


i = 4  Accuracy  0.5


<ipython-input-32-3917d4d5ed91>:15: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if token in neg_words:


i = 5  Accuracy  0.5


<ipython-input-32-3917d4d5ed91>:15: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if token in neg_words:


i = 6  Accuracy  0.5


Все сломалось(( со словарями сложно  
В итоге самая лучшая точность получилась просто подбром окна с исключением стоп-слов (0.7972972972972973), в предыдущей итерации  без (0.7905405405405406)